# GPUs

In [1]:
!nvidia-smi

Mon Aug  1 14:55:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   64C    P0    67W / 250W |   2090MiB / 23040MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   

## 1. Device

PyTorch's calculation will try to use all CPU cores. However, a `gpu` device only represents one card and the corresponding memory. If there are multiple GPUs, we use `torch.device(f'cuda:{i}')` to represent the $i^{th}$ gpu

In [3]:
import torch 
from torch import nn 
from d2l import torch as d2l

def cpu(): #@save
    return torch.device('cpu')

def gpu(i=0): #@save
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [6]:
def num_gpus(): #@save
    return torch.cuda.device_count()
num_gpus()

2

Define two convenient functions that allow us to run code even if the requested GPUs do not exist

In [8]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [try_gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

## 2. Tensors and GPUs
By default, tensors are created on the CPU. We can query the device where the tensor is located.

In [10]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

### 2.1 Storage on the GPU

In [11]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [12]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8234, 0.6525, 0.5370],
        [0.0277, 0.8811, 0.4176]], device='cuda:1')

### 2.2 Copying
To perform operation of two tensors, we need to decide the device. The two tensors should be on the same device.

In [13]:
try: 
    X + Y
except:
    print(f'Tensor in {X.device} cannot add with tensor in {Y.device}')

Tensor in cuda:0 cannot add with tensor in cuda:1


In [18]:
# copy X to Z in device cuda: 1
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


Y and Z are on the same GPU so we can add them up.

In [19]:
Y + Z

tensor([[1.8234, 1.6525, 1.5370],
        [1.0277, 1.8811, 1.4176]], device='cuda:1')

If Z is already on the cuda: 1, `Z.cuda(1)` will return Z instead of making a copy

In [22]:
Z.cuda(1) is Z

True

## 3. Neural Networks and GPUs

In [44]:
net = nn.Sequential(nn.LazyLinear(1))
net.to(device=try_gpu())

/home/wangzixiao/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential(
  (0): LazyLinear(in_features=0, out_features=1, bias=True)
)

When the input is a tensor on the GPU, the model will calculate the result on the same GPU.

In [45]:
net(X)

tensor([[-0.2743],
        [-0.2743]], device='cuda:0', grad_fn=<AddmmBackward0>)

Let's confirm that the model parameters are stored on the same GPU

In [46]:
net[0].weight.data.device

device(type='cuda', index=0)

Let the trainer support GPU

In [47]:
@d2l.add_to_class(d2l.Trainer)
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)
def prepare_batch(self, batch):
    # if there is gpu
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model